<a href="https://colab.research.google.com/github/bsong75/brendensong.github.io/blob/main/anomaly_4simple%20python%20methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
warnings.filterwarnings('ignore')
df =  pd.read_csv("/content/sample_data/traffic.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      32 non-null     object
 1   Sessions  32 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 640.0+ bytes


In [3]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
px.line(data_frame=df, x='Date',y='Sessions',title="Lets view outlier")

In [7]:
px.box(data_frame=df,x='Sessions')

In [8]:
px.violin(data_frame=df,x='Sessions')

In [9]:
# Using IQR to estimate outliers

def traditional_outlier(df,x):
    q1  =  df[x].quantile(.25)
    q3  =  df[x].quantile(.75)
    iqr = q3-q1
    df['Traditional']  = np.where(df[[x]]<(q1-1.5*iqr),-1,
                         np.where(df[[x]]>(q3+1.5*iqr),-1,1))
    return df
traditional_outlier(df, 'Sessions')

,Date,Sessions,Traditional
0,2022-05-15,663,1
1,2022-05-16,1627,1
2,2022-05-17,1630,1
3,2022-05-18,1666,1
4,2022-05-19,1604,1
5,2022-05-20,1188,1
6,2022-05-21,435,1
7,2022-05-22,726,1
8,2022-05-23,1565,1
9,2022-05-24,1649,1


In [10]:
#Isolation Forest
from sklearn.ensemble import IsolationForest

In [11]:
IsolationForest().fit(df[['Sessions']]).predict(df[['Sessions']])

array([ 1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1,
        1, -1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1])

In [12]:
df['Isolation Forest'] = IsolationForest().fit(df[['Sessions']]).predict(df[['Sessions']])

In [13]:
from sklearn.covariance import EllipticEnvelope
EllipticEnvelope().fit(df[['Sessions']]).predict(df[['Sessions']])

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1,
        1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [14]:
from sklearn.neighbors import LocalOutlierFactor
LocalOutlierFactor(n_neighbors=5, novelty=True).fit(df[['Sessions']]).predict(df[['Sessions']])

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1,  1,
        1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [15]:
def outliers_find(df,x):
    df['Local Outlier'] = LocalOutlierFactor(n_neighbors=5, novelty=True).fit(df[[x]]).predict(df[[x]])
    df['Isolation Forest'] = IsolationForest().fit(df[[x]]).predict(df[[x]])
    df['Elliptical'] = EllipticEnvelope().fit(df[[x]]).predict(df[[x]])
    return df

In [17]:
outliers_find(df, 'Sessions')

,Date,Sessions,Traditional,Isolation Forest,Local Outlier,Elliptical
0,2022-05-15,663,1,1,1,1
1,2022-05-16,1627,1,1,1,1
2,2022-05-17,1630,1,1,1,1
3,2022-05-18,1666,1,1,1,1
4,2022-05-19,1604,1,1,1,1
5,2022-05-20,1188,1,1,1,1
6,2022-05-21,435,1,-1,1,1
7,2022-05-22,726,1,1,1,1
8,2022-05-23,1565,1,1,1,1
9,2022-05-24,1649,1,1,1,1


In [ ]:
def outliers_find(df,x):
    q1  =  df[x].quantile(.25)
    q3  =  df[x].quantile(.75)
    iqr = q3-q1
    df['Traditional']  = np.where(df[[x]]<(q1-1.5*iqr),-1,
                         np.where(df[[x]]>(q3+1.5*iqr),-1,1))
    df['Local Outlier'] = LocalOutlierFactor(n_neighbors=5, novelty=True).fit(df[[x]]).predict(df[[x]])
    df['Isolation Forest'] = IsolationForest().fit(df[[x]]).predict(df[[x]])
    df['Eliptic Envelope'] =EllipticEnvelope().fit(df[['Sessions']]).predict(df[['Sessions']])
    return df